In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
import shutil
from pathlib import Path
from sorted_alpha import sorted_alpha
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from scipy import interpolate
from frame_count import frame_count
from scipy.signal import savgol_filter
from scipy.spatial import distance


In [ ]:
def get_skeletons(frame_no, detection, track1, track2):
    knn =  NearestNeighbors(n_neighbors=1, metric='euclidean')
    """function that reads a detections frame, checks if necks are present, and returns the 2 closest skeletons from previous detections (in case 3 have been detected)"""
    data = pd.read_csv(detection)
    data = data.fillna(0)
    ## first check if it contains the neck 
    temp_data = data
    for i in range(0, len(data.columns[1:]),3):
        if data[data.columns[1:][i]].loc[1] == 0:
            temp_data = temp_data.drop([data.columns[1:][i],data.columns[1:][i+1], data.columns[1:][i+2]], axis=1) ## drop skeletons with missing neck
    data=temp_data
    if len(data.columns) < 4: ## if not enough skeletons with neck present return 0 
        return(None)
    elif frame_no ==0: 
        if len(data.columns)>4:  ### when first starting tracking initialise with either one or two skeletons 
            neck_1 = [data['x_1'].loc[1], data['y_1'].loc[1]]
            neck_2 = [data['x_2'].loc[1], data['y_2'].loc[1]]
            return([neck_1, neck_2])
        else: 
            neck_1 = [data['x_1'].loc[1], data['y_1'].loc[1]]
            neck_2 = [0,0]
            return([neck_1, neck_2])
    else:
        if len(data.columns)> 10: ### drop the fourth skeleton if it has been detected
            data = data.drop(data.columns[10:], axis=1) 
        ## trying to tell which one is neck 1 and which is neck 2 

        if track2 == []:         ## if track2 isn't initialized yet:
            detections_x = [data[data.columns[1:][col]].loc[1] for col in  range(0, len(data.columns[1:]), 3)]; detections_y = [data[data.columns[2:][col]].loc[1] for col in  range(0, len(data.columns[2:]), 3)]
            detections = np.vstack([detections_x, detections_y]).T
            probs = [data[data.columns[3:][col]].loc[1] for col in  range(0, len(data.columns[3:]), 3)]
            ## fit knn with the current detections and see which is closer to track1.lastitem
            knn.fit(detections); closest_index = knn.kneighbors([track1[-1]])[1];
            neck_1 = detections[closest_index][0][0]
            detections = np.delete(detections, closest_index, axis=0)
            probs = np.delete(probs, closest_index)
            if detections.size !=0:  ## ensuring that if there was just one skeleton there isn't a point in doing any comparisons as neck_2 will be empty
                neck_2 = detections[np.argmax(probs)]
            else:
                neck_2 = [0,0]
            return([neck_1,neck_2])
        else:
            
            detections_x = [data[data.columns[1:][col]].loc[1] for col in  range(0, len(data.columns[1:]), 3)]; detections_y = [data[data.columns[2:][col]].loc[1] for col in  range(0, len(data.columns[2:]), 3)]
            detections = np.vstack([detections_x, detections_y]).T
            
            probs = [data[data.columns[3:][col]].loc[1] for col in  range(0, len(data.columns[3:]), 3)]
            ## fit knn with the current detections and see which is closer to track1.lastitem
            if len(detections)>1:
                knn.fit(detections); closest_index = knn.kneighbors([track1[-1]])[1];
                neck_1 = detections[closest_index[0][0]]
                detections= np.delete(detections, closest_index, axis=0)
                probs = np.delete(probs, closest_index)
                ## get the closest neck to last detection in track2
                knn.fit(detections); closest_index = knn.kneighbors([track2[-1]])[1]
                neck_2 = detections[closest_index[0][0]]
         
                return([neck_1, neck_2])
            else:
                return([detections[0],[0,0]]) 
                

        
    
        

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def track_skeletons(frame_no, track1, track2, knn):
    data = pd.read_csv(detection)
    data = data.fillna(0)
    ## first check if it contains the neck 
    temp_data = data.copy()
    for i in range(0, len(data.columns[1:]), 3):
        if data[data.columns[1:][i]].loc[1] == 0:
            temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1) ## drop skeletons with missing neck
    data = temp_data
    if len(data.columns) < 4: ## if not enough skeletons with neck present return None 
        return None
    elif frame_no == 0: 
        if len(data.columns) > 4:  ### when first starting tracking initialize with either one or two skeletons 
            skeleton_1 = data.iloc[:, 1:4].values.flatten().tolist()
            skeleton_2 = data.iloc[:, 4:7].values.flatten().tolist()
            return [skeleton_1, skeleton_2]
        else: 
            skeleton_1 = data.iloc[:, 1:4].values.flatten().tolist()
            skeleton_2 = [0] * len(skeleton_1)
            return [skeleton_1, skeleton_2]
    else:
        if len(data.columns) > 10: ### drop the fourth skeleton if it has been detected
            data = data.drop(data.columns[10:], axis=1) 
        
        ## trying to tell which one is neck 1 and which is neck 2 
        if not track2:  ## if track2 isn't initialized yet:
            detections = []
            for col in range(0, len(data.columns[1:]), 3):
                skeleton = data.iloc[:, col:col+3].values.flatten().tolist()
                detections.append(skeleton)
            detections = np.array(detections)
            necks = detections[:, :2]  # Extract neck keypoints
            probs = detections[:, 2]  # Extract probabilities
            
            ## fit knn with the current detections and see which is closer to track1.lastitem
            knn.fit(necks)
            closest_index = knn.kneighbors([track1[-1][:2]])[1][0][0]
            skeleton_1 = detections[closest_index].tolist()
            detections = np.delete(detections, closest_index, axis=0)
            probs = np.delete(probs, closest_index)
            if detections.size != 0:  ## ensuring that if there was just one skeleton there isn't a point in doing any comparisons as skeleton_2 will be empty
                skeleton_2 = detections[np.argmax(probs)].tolist()
            else:
                skeleton_2 = [0] * len(skeleton_1)
            return [skeleton_1, skeleton_2]
        else:
            detections = []
            for col in range(0, len(data.columns[1:]), 3):
                skeleton = data.iloc[:, col:col+3].values.flatten().tolist()
                detections.append(skeleton)
            detections = np.array(detections)
            necks = detections[:, :2]  # Extract neck keypoints
            probs = detections[:, 2]  # Extract probabilities
            
            ## fit knn with the current detections and see which is closer to track1.lastitem and track2.lastitem
            knn.fit(necks)
            closest_index_1 = knn.kneighbors([track1[-1][:2]])[1][0][0]
            skeleton_1 = detections[closest_index_1].tolist()
            detections = np.delete(detections, closest_index_1, axis=0)
            probs = np.delete(probs, closest_index_1)
            
            if detections.size != 0:
                closest_index_2 = knn.kneighbors([track2[-1][:2]])[1][0][0]
                skeleton_2 = detections[closest_index_2].tolist()
            else:
                skeleton_2 = [0] * len(skeleton_1)
            return [skeleton_1, skeleton_2]

In [ ]:
def relative_position(track1, track2, new_detection1, new_detection2):
    x_difference = []
    y_difference = []
    for detect1, detect2 in zip(track1[-50:-1], track2[-50:-1]): # the last 50 detections
        for index,coords in enumerate(zip(detect1, detect2)):
            if index%2 ==0:
                x_difference.append(coords[0]-coords[1])
            else:
                y_difference.append(coords[0]-coords[1])
    if np.mean(np.abs(x_difference)) > np.mean(np.abs(y_difference)): ### do they tend to be farther on the x axis? 
        if np.mean(x_difference) > 0: # is tracklet1 farther on x axis than tracklet2? relative position
            if new_detection1[0] - new_detection2[0] > 0: # is detection 1 farther on x axis than detection 2? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
        elif np.mean(x_difference) <0:  # is tracklet2 farther on x axis than tracklet1?
            if new_detection1[0] - new_detection2[0] < 0: # is detection 2 farther on x axis than detection 1? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
    elif np.mean(y_difference) > 0:
        if new_detection1[1] - new_detection2[1] > 0: # is detection 1 farther on y axis than detection 2? i.e. does new detection converge with past history 
            return(1)
        else:

            return(2)
    elif np.mean(y_difference) < 0:  # is tracklet2 farther on y axis than tracklet1?
        if new_detection1[1] - new_detection2[1] < 0:
            return(1)
        else:

            return(2)        ### do they tend to be farther on the y axis? 
       

In [ ]:
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int((x.split("."))[0].split("_")[1]) for x in detects]

In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def track_skeletons(frame_no, data, track1, track2, knn):
    # Check if it contains the neck keypoint
    temp_data = data.copy()
    for i in range(0, len(data.columns[1:]), 3):
        if data[data.columns[1:][i]].loc[1] == 0:
            temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1)  # Drop skeletons with missing neck
    data = temp_data
    
    if len(data.columns) < 4:  # If not enough skeletons with neck present, return None
        return None
    
    # Extract entire skeletons and neck keypoints
    detections = []
    neck_keypoints = []
    for col in range(0, len(data.columns[1:]), 3):
        skeleton = data.iloc[:, col:col+3].values.flatten().tolist()
        detections.append(skeleton)
        neck_keypoints.append(skeleton[3:6])  # Assuming neck keypoint is the second row (index 1)
    detections = np.array(detections)
    neck_keypoints = np.array(neck_keypoints)
    
    if frame_no == 0:
        if len(detections) > 1:
            return [detections[0].tolist(), detections[1].tolist()]
        else:
            return [detections[0].tolist(), [0] * len(detections[0])]
    
    if track2 == []:  # If track2 isn't initialized yet
        knn.fit(neck_keypoints)
        closest_index = knn.kneighbors([track1[-1][3:6]])[1][0][0]  # Compare only neck keypoints
        skeleton_1 = detections[closest_index].tolist()
        detections = np.delete(detections, closest_index, axis=0)
        neck_keypoints = np.delete(neck_keypoints, closest_index, axis=0)
        
        if detections.size != 0:
            probs = detections[:, 2::3].max(axis=1)  # Extract probabilities
            skeleton_2 = detections[np.argmax(probs)].tolist()
        else:
            skeleton_2 = [0] * len(skeleton_1)
        
        return [skeleton_1, skeleton_2]
    else:
        knn.fit(neck_keypoints)
        closest_index_1 = knn.kneighbors([track1[-1][3:6]])[1][0][0]  # Compare only neck keypoints
        skeleton_1 = detections[closest_index_1].tolist()
        detections = np.delete(detections, closest_index_1, axis=0)
        neck_keypoints = np.delete(neck_keypoints, closest_index_1, axis=0)
        
        if detections.size != 0:
            closest_index_2 = knn.kneighbors([track2[-1][3:6]])[1][0][0]  # Compare only neck keypoints
            skeleton_2 = detections[closest_index_2].tolist()
        else:
            skeleton_2 = [0] * len(skeleton_1)
        
        return [skeleton_1, skeleton_2]

In [ ]:
# 1. read data
data = pd.read_csv('/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal/frame_1.csv')
temp_data = data.copy()
frame_no = 0
# 2. drop skeletons with missing neck
for i in range(0, len(data.columns[1:]), 3):
    if data[data.columns[1:][i]].loc[1] == 0:
        temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1)  # Drop skeletons with missing neck
data = temp_data

# 3. if no skeletons with neck return None
if len(data.columns) < 4:  # If not enough skeletons with neck present, return None
    print('none')

# remove fourth skeleton if detected
elif  len(data.columns)> 10:
    ### drop the fourth skeleton if it has been detected
    data = data.drop(data.columns[10:], axis=1) 

# 4. initialize the tracking with the first two skeletons and save the neck_points

detections = []
neck_keypoints = []
for col in range(0, len(data.columns[1:]), 3):
    skeleton = data.iloc[:, col:col+3].values
    detections.append(skeleton[:,1:])
    neck_keypoints.append(skeleton[:,1:][1])  # Assuming neck keypoint is the second row (index 1)
detections = np.array(detections) # Shape: (num_skeletons, num_keypoints)
neck_keypoints = np.array(neck_keypoints)

#return(detections, neck_keypoints)


In [ ]:
neck_keypoints

In [ ]:
import pandas as pd
import numpy as np

def process_skeleton_data(csv_file_path):
    # 1. read data
    data = pd.read_csv(csv_file_path)
    temp_data = data.copy()

    
    # 2. drop skeletons with missing neck
    for i in range(0, len(data.columns[1:]), 3):
        neck_value = data[data.columns[1:][i]].loc[1]
        if pd.isna(neck_value) or neck_value == 0:
            temp_data = temp_data.drop([data.columns[1:][i], data.columns[1:][i+1], data.columns[1:][i+2]], axis=1)  # Drop skeletons with missing neck
    data = temp_data

    # 3. if no skeletons with neck return None
    if len(data.columns) < 4:  # If not enough skeletons with neck present, return None
        return None, None

    # remove fourth skeleton if detected
    elif len(data.columns) > 10:
        # Drop the fourth skeleton if it has been detected
        data = data.drop(data.columns[10:], axis=1) 

    # 4. initialize the tracking with the first two skeletons and save the neck_points
    detections = []
    neck_keypoints = []
    for col in range(0, len(data.columns[1:]), 3):
        skeleton = data.iloc[:, col:col+3].values
        detections.append(skeleton[:,1:])  # Select only x and y columns
        neck_keypoints.append(skeleton[:,1:][1])  # Assuming neck keypoint is the second row (index 1)
    
    detections = np.array(detections)  # Shape: (num_skeletons, num_keypoints, 2)
    neck_keypoints = np.array(neck_keypoints)  # Shape: (num_skeletons, 2)

    return detections, neck_keypoints

# Example usage
csv_file_path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal/frame_796.csv'
detections, neck_keypoints = process_skeleton_data(csv_file_path)

print(neck_keypoints)

In [ ]:
def relative_position(track1, track2, new_detection1, new_detection2):
    x_difference = []
    y_difference = []
    for detect1, detect2 in zip(track1[-50:-1], track2[-50:-1]): # the last 50 detections
        for index,coords in enumerate(zip(detect1, detect2)):
            if index%2 ==0:
                x_difference.append(coords[0]-coords[1])
            else:
                y_difference.append(coords[0]-coords[1])
    if np.mean(np.abs(x_difference)) > np.mean(np.abs(y_difference)): ### do they tend to be farther on the x axis? 
        if np.mean(x_difference) > 0: # is tracklet1 farther on x axis than tracklet2? relative position
            if new_detection1[0] - new_detection2[0] > 0: # is detection 1 farther on x axis than detection 2? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
        elif np.mean(x_difference) <0:  # is tracklet2 farther on x axis than tracklet1?
            if new_detection1[0] - new_detection2[0] < 0: # is detection 2 farther on x axis than detection 1? i.e. does new detection converge with past history 
                return(1)
            else:
                return(2)
    elif np.mean(y_difference) > 0:
        if new_detection1[1] - new_detection2[1] > 0: # is detection 1 farther on y axis than detection 2? i.e. does new detection converge with past history 
            return(1)
        else:

            return(2)
    elif np.mean(y_difference) < 0:  # is tracklet2 farther on y axis than tracklet1?
        if new_detection1[1] - new_detection2[1] < 0:
            return(1)
        else:

            return(2)        ### do they tend to be farther on the y axis? 
       

In [ ]:
import numpy as np

def relative_position(track1, track2, new_detection1, new_detection2):
    x_diff = [d1 - d2 for d1, d2 in zip(track1[-50:-1:2], track2[-50:-1:2])]
    y_diff = [d1 - d2 for d1, d2 in zip(track1[-49::2], track2[-49::2])]

    mean_x_diff = np.mean(x_diff)
    mean_y_diff = np.mean(y_diff)

    if abs(mean_x_diff) > abs(mean_y_diff):
        if mean_x_diff > 0:
            return 1 if new_detection1[0] > new_detection2[0] else 2
        else:
            return 1 if new_detection1[0] < new_detection2[0] else 2
    else:
        if mean_y_diff > 0:
            return 1 if new_detection1[1] > new_detection2[1] else 2
        else:
            return 1 if new_detection1[1] < new_detection2[1] else 2

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

# Initialize tracking variables
tracks = []
frames = []
knn = NearestNeighbors(n_neighbors=1, metric='euclidean')

# Example usage
path = '/Users/andrei-macpro/Documents/Data/pose/meal_openpose/1043_meal'
detects = sorted_alpha(path)
numbers = [int(f.split('_')[-1].split('.')[0]) for f in detects]

for i, number in zip(range(len(detects)), numbers):
    frames.append(number)
    print('frame no:', number)
    
    data = pd.read_csv(os.path.join(path, detects[i]))
    
    if i == 0:  # For the first frame, just get the skeletons, don't do anything
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Initialize track lists for each detection
        for _ in range(len(detections)):
            tracks.append([])

        # Append each detection to its corresponding track list
        for j, detection in enumerate(detections):
            tracks[j].append(detection)

    else:
        detections, necks = process_skeleton_data(os.path.join(path, detects[i]))
        # Get the last neck positions from the tracks
        last_necks = np.array([track[-1][1] for track in tracks])

        # Fit the KNN model on the last neck positions
        knn.fit(last_necks)
        # Keep track of which detections have been assigned
        assigned_detections = set()

        # Store distances and indices
        distances_indices = []
        # Compute Euclidean distances between each pair of new detections 
        necks_array = np.array(necks)
        num_detections = len(necks_array)
        distances = np.zeros((num_detections, num_detections))


        for j in range(num_detections):
            for k in range(j + 1, num_detections):
                dist = euclidean(necks_array[j], necks_array[k])
                distances[j, k] = dist
                distances[k, j] = dist  # Symmetric matrix
        # Check if distance between the first 2 detections are more than 200
        if num_detections > 1:
            if i <50 or np.any(distances[0:2,0:2] > 200:
                # Find the closest track for each new detection
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    distances_indices.append((dist[0][0], index[0][0], j))

                # Sort detections by distance to the closest track
                distances_indices.sort()

                # Assign the closest detections to the tracks
                for dist, closest_track_index, j in distances_indices[:len(tracks)]:
                    tracks[closest_track_index].append(detections[j])
                    assigned_detections.add(j)

                # If there are more detections than tracks, create new tracks for the remaining detections
                for dist, closest_track_index, j in distances_indices[len(tracks):]:
                    new_track_index = len(tracks)
                    tracks.append([])  # Initialize a new track
                    tracks[new_track_index].append(detections[j])
                    assigned_detections.add(j)
            else:
                relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                if relative_position == 1:
                    tracks[0].append(detections[0])
                    tracks[1].append(detections[1])
                else:
                    tracks[0].append(detections[1])
                    tracks[1].append(detections[0])
                
        if num_detections == 1:
            if np.any(distances) > 200:
            # this should still do the relative position calculation if distance<200 
                for j, neck in enumerate(necks):
                    dist, index = knn.kneighbors([neck])
                    tracks[index[0][0]].append(detections[j])
                    assigned_detections.add(j)
            else:
                if i <50:
                    for j, neck in enumerate(necks):
                        dist, index = knn.kneighbors([neck])
                        tracks[index[0][0]].append(detections[j])
                        assigned_detections.add(j)

                   ## think about this : what if there's only one detection and i>50? 
                else:
                    relative_position = relative_position(tracks[0], tracks[1], necks[0], necks[1])
                    if relative_position == 1:
                        tracks[0].append(detections[0])
                        tracks[1].append(detections[1])
                    else:
                        tracks[0].append(detections[1])
                        tracks[1].append(detections[0])



### the case where num_detections > 1 and distances <200  and i < 50 is not handled yet


                

        else:
            # if number of detection is less than 2 just assign the detections to the closest tracks
           




In [ ]:
np.any(distances[0:2,0:2] > 200)

In [ ]:
distances[0:2,0:2]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os


        if skeletons is not None:
            skeleton1 = np.array(skeletons[0])
            skeleton2 = np.array(skeletons[1])
        else:
            continue
        if skeleton2[0] > 0:
            track1.append(skeleton1); frames1.append(number)
            track2.append(skeleton2); frames2.append(number)
        else:
            track1.append(skeleton1); frames1.append(number)
        continue
    
    skeletons = track_skeletons(i, data, track1, track2, knn=knn)
    if skeletons is not None:
        skeleton1 = skeletons[0]
        skeleton2 = skeletons[1]
    else:
        continue
    neck1 = skeleton1[3:6]
    neck2 = skeleton2[3:6]
    if len(track1) < 100 or len(track2) < 100:
    # If only one skeleton has been detected
        if skeleton2[0] == 0:  # If new detection only has one skeleton
            if track2 == []:  # If second tracking hasn't been initialized
                if np.linalg.norm(track1[-1][3:6] - neck1) < threshold:
                    track1.append(skeleton1); frames1.append(number)
                else:
                    track2.append(skeleton1); frames2.append(number)
            else:
                if np.linalg.norm(track1[-1][3:6] - neck1) < np.linalg.norm(track2[-1][3:6] - neck1):
                    track1.append(skeleton1); frames1.append(number)
                else:
                    track2.append(skeleton1); frames2.append(number)
        else:
            if np.linalg.norm(track1[-1][3:6] - neck1) < np.linalg.norm(track1[-1][3:6] - neck2):
                track1.append(skeleton1); frames1.append(number)
                track2.append(skeleton2); frames2.append(number)
            else:
                track1.append(skeleton2); frames1.append(number)
                track2.append(skeleton1); frames2.append(number)